# Day 2: Ollama Solution for Website Summarization
### Building and Deploying Website Summarization Tools with Ollama

In [14]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

url = "https://sitemakerlab.com/" 
site = Website(url)
messages = messages_for(site)

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")


**Summary of Website**
=========================
### Overview

The "Home - Edward Donner" website is a personal blog hosted by Edward Donner, the co-founder and CTO of Nebula.io. The website shares insights on his professional experiences with AI, particularly in applying LLMs (Large Language Models) to help people discover their potential.

### Recent Developments
------------------------

* **Courses**: Upcoming online courses, including "Connecting my courses – become an LLM expert and leader" and "The Complete Agentic AI Engineering Course"
* **Events**:
	+ May 28, 2025: Launch connecting his courses (LLM Expert and Leader)
	+ May 18, 2025: 2025 AI Executive Briefing
	+ April 21, 2025: AI Executive Briefing
	+ January 23, 2025: The Complete Agentic AI Engineering Course

### About the Creator
-------------------------

* Edward Donner is the co-founder and CTO of Nebula.io.
* He has experience as a founder and CEO of an AI startup that was acquired in 2021.
* His interests include DJing, amateur electronic music production, and reading Hacker News.

### Contact Information
-------------------------

* Email: [ed @ edwarddonner . com](mailto:ed@edwarddonner.com)
* Website: www.edwarddonner.com
* Social media links:
	+ LinkedIn
	+ Twitter
	+ Facebook